In [1]:
import json
import pandas as pd
import plotly.express as px
import numpy as np

In [4]:
with open('../data/docs.json', encoding = "utf8") as f:
    docs = json.load(f)

# Mots par document (sans ponctuation, sans stop-words)

In [5]:
nbDocs = len(docs)
tailleDocs = [len(doc) for doc in docs]
nbMots = sum(tailleDocs)

print(f"Nombre de documents : {nbDocs} | Nombre de mots : {nbMots}")

Nombre de documents : 9501 | Nombre de mots : 71334


In [6]:
tailleDocs = pd.DataFrame(tailleDocs, columns = ["Lemme"])

fig = px.histogram(tailleDocs, x = "Lemme", title = "Histogramme du nombre de mots par documents")
fig.show()

# Importance des lemmes

## Présence globale (fréquence d'apparition sur le corpus)

In [7]:
freqMots = {}
ensembleMots = set()
for doc in docs:
    for mot in doc:
        if mot not in freqMots:
            ensembleMots.add(mot)
            freqMots[mot] = 1
        else:
            freqMots[mot] += 1

Enregistrement des lemmes (uniques) dans un fichier texte

In [8]:
file = open("../data/liste_lemmes.txt", 'w')
for lemme in freqMots.keys():
    file.writelines(lemme + '\n')
file.close()

Ordonnancement décroissant, transformation en _dataframe_

In [132]:
freqMots = sorted(freqMots.items(), key = lambda m: m[1], reverse = True)
freqMots = pd.DataFrame(freqMots, columns = ['Lemme', 'Fréquence'])
freqMots['Fréquence'] *= 100 / nbMots

### Lemmes les plus utilisés (représentatifs du sujet)

In [133]:
nbLemmesAfficher = 10
fig = px.bar(freqMots.iloc[:nbLemmesAfficher,:], x = 'Lemme', y = 'Fréquence',
title = "Fréquence (%) d'apparition des lemmes dans les propositions")
fig.show()

### Lemmes les moins utilisés (les plus discriminants)

In [134]:
freqMin = 100 / nbMots
motsUniques = [freqMots.loc[i,"Lemme"] for i in range(len(freqMots)) if freqMots.loc[i,"Fréquence"] == freqMin]
del freqMin

In [135]:
print(f"Nombre ; Proportion de mots n'ayant été utilisés qu'une seule fois sur l'ensemble des documents : {len(motsUniques)} ; {100 * len(motsUniques) / len(freqMots)} %.")

Nombre ; Proportion de mots n'ayant été utilisés qu'une seule fois sur l'ensemble des documents : 3467 ; 45.53454163383241 %.


## Termes discriminants (IDF, nombre de documents possédant chaque mot)

In [136]:
idf = {}
for mot in ensembleMots:
    nbDocsMotPresent = 0
    for doc in docs:
        if mot in doc:
            nbDocsMotPresent += 1
    idf[mot] = np.log(nbDocs / nbDocsMotPresent)

Ordonnancement décroissant, transformation en _dataframe_

In [137]:
idf = sorted(idf.items(), key = lambda m: m[1], reverse = True)
idf = pd.DataFrame(idf, columns = ['Lemme', 'IDF'])

In [138]:
#nbLemmesAfficher = 20
fig = px.bar(idf.loc[:,:], x = 'Lemme', y = 'IDF',
title = "IDF des différents lemmes")
fig.show()